<a href="https://colab.research.google.com/github/NotYeshwanthReddy/empty/blob/master/Fifa19_Rapido_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

# Player replacement 


# Fifa 19

In [1]:
!pip install factor_analyzer

Imports

In [0]:
import pandas as pd   # reading csv file
import numpy as np    # manipulating data
from sklearn.cluster import KMeans    # Kmeans model
import matplotlib.pyplot as plt       # plotting graphs
from google.colab import drive        # import data from drive

from factor_analyzer import FactorAnalyzer  # decrease data dimensionality

Mounting the Dataset

In [3]:
drive.mount('/content/gdrive')    # mounting drive storage

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Reading the Dataset

In [4]:
data = pd.read_csv('/content/gdrive/My Drive/Projects/Fifa19/data.csv', delimiter=',', header=0, index_col=0)
# data = pd.read_csv('data.csv', delimiter=',', header=0, index_col=0)
print(data.head())

       ID               Name  Age  ... GKPositioning GKReflexes Release Clause
0  158023           L. Messi   31  ...          14.0        8.0        €226.5M
1   20801  Cristiano Ronaldo   33  ...          14.0       11.0        €127.1M
2  190871          Neymar Jr   26  ...          15.0       11.0        €228.1M
3  193080             De Gea   27  ...          88.0       94.0        €138.6M
4  192985       K. De Bruyne   27  ...          10.0       13.0        €196.4M

[5 rows x 88 columns]


In [0]:
# Taking player names in separate array for future use.
names = list(data[['Name']].values)
# Taking only requirefd features (Dribbling, GoalKeeping, Shooting...)
data = data.iloc[:, 55:87]
print(data.shape)
# Removing Null Values
data = data.dropna(how='any',axis=0) 
print(data.shape)
# Print Head
print(data.head())

In [0]:
# printing the selected column names.
col = list(data.columns)
for i in range(len(col)):
  print(i, col[i])

In [0]:
# Corellating the data for analysis
corr = data.corr()

In [0]:
# Plotting the Heatmap
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(corr,cmap='coolwarm', vmin=-1, vmax=1)
fig.colorbar(cax)
ticks = np.arange(0,len(data.columns),1)
ax.set_xticks(ticks)
plt.xticks(rotation=90)
ax.set_yticks(ticks)
ax.set_xticklabels(data.columns)
ax.set_yticklabels(data.columns)
# Increasing the size of figure
fig.set_size_inches(18.5, 10.5)
plt.show()

From the above figure, we can see many features are corellated to eachother in positive or negative way. The features which have high corellation factor can be merged into one feature which will help in decreasing the dimensionality of the problem.<br>
We are using Factor analysis technique to decrease the dimensions of our data based on the eigen values.

# Factor analysis
The objective of factor analysis is to reduce the number of observed variables and find unobservable variables.
Before we perform factor analysis, we need to evaluate the “factorability” of our dataset. Factorability means "can we found the factors in the dataset?". There are two methods to check the factorability or sampling adequacy are as follows

In [0]:
# Bartlett’s Test 
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
chi_square_value,p_value=calculate_bartlett_sphericity(data)
chi_square_value, p_value

(952204.763829223, 0.0)

In this Bartlett ’s test, the p-value is 0. The test was statistically significant, indicating that the observed correlation matrix is not an identity matrix.

In [0]:
# Kaiser-Meyer-Olkin Test
from factor_analyzer.factor_analyzer import calculate_kmo
kmo_all,kmo_model=calculate_kmo(data)
print(kmo_model)

The overall KMO for our data is ~0.97, which is excellent. This value indicates that you can proceed with your planned factor analysis.

In [0]:
# Creating factor analysis object and performing factor analysis
import factor_analyzer
factors = 6
fa = FactorAnalyzer(n_factors=factors,rotation="varimax")
fa.fit(data)
# Check Eigenvalues
ev, v = fa.get_eigenvalues()
print(ev)

The array above consists of the eigen values of each feature.

In [0]:
# computing corellation coefficients between all factors and all features.
loadings = fa.loadings_
print(pd.DataFrame.from_records(loadings))

Here we can see that<br>
Factor 0 has high factor loadings for [1, 2, 3, 4, 5, 6, 7, 11, 13, 17, 20, 21, 22, 23]<br>
Factor 1 has high factor loadings for [0, 27, 28, 29, 30, 31].<br>
Factor 2 has high factor loadings for [15, 18, 19, 24, 25, 26].<br>
Factor 3 has high factor loadings for [8, 9, 10].<br>
Factor 4 has high factor loadings for [12, 16].<br>
Factor 5 has high factor loadings for [14].<br>



## Merging features using average
Each factor is having high corellation with multiple features. So, we are taking the average of all those features and converting it into one new feature.

In [0]:
# Taking it into an array
df = pd.DataFrame.from_records(loadings)
# Squaring the valus to get rid of negative values
df = np.square(df)
# Taking the max value index of each row
df = df.idxmax(axis=1)
arr = [[] for _ in range(6)]
for i in range(df.size):
  arr[df[i]].append(i)
print(arr)

The above array describes which features needs to be merged(averaged).

In [0]:
# Renaming the columns from ['BallControl', 'Dribbling', 'Accuracy', ...] to [0,1,2,3,...]
data.columns = [i for i in range(data.shape[1])]
# empty pandas dataframe
new_data = pd.DataFrame()
# Averaging the columns and adding them into new_data
for i in range(len(arr)):
  temp = data[arr[i]]
  new_data[i] = temp.mean(axis=1)
# new_data is the dataframe of new features (after averaging)
print(new_data.head)

# Kmeans

In [0]:
# Converting dataframe into required format
input = list(new_data.values)
print(input)

In [0]:
# Calculating the cost for different no. of clusters
cost =[] 
# calculatig for 1-50 clusters
for i in range(1, 50): 
    KM = KMeans(n_clusters = i, max_iter = 500) 
    KM.fit(input) 
    cost.append(KM.inertia_) 

In [0]:
# Plotting the cost function
plt.plot(range(1, 50), cost, color ='g', linewidth ='3') 
plt.xlabel("Value of K") 
plt.ylabel("Sqaured Error (Cost)") 
plt.show()

As we can see that here is no much change in the error from 15 but if we take 15 clusters the number of players for each cluster will be more. So we are using 35 clusters which is optimal.

In [0]:
kmeans = KMeans(n_clusters=35, random_state=0).fit(input)

In [0]:
lab = list(kmeans.labels_)

In [0]:
data = pd.read_csv('/content/gdrive/My Drive/Projects/Fifa19/data.csv', delimiter=',', header=0, index_col=0)
names = list(data[['Name']].values)
data = data.iloc[:, 55:87]
data['Name'] = names
data = data.dropna(how='any',axis=0)

In [0]:
# Searching replacement for the player below
player_name = "L. Messi"



# Selecting the row which has our player name
player_data = data.loc[data.Name == player_name]
# creating empty array
player = np.array([])
# calculating the features of the player (using averaging)
player_data.columns = [i for i in range(player_data.shape[1])]
for i in range(len(arr)):
  temp = player_data[arr[i]]
  player = np.append(player, temp.mean(axis=1))
  
# print(player)
# Finding the players cluster
output_classs = kmeans.predict([player]) 
# Printing the cluster number
print("Class : ",output_classs)

In [0]:
# Output all the players in that Cluster.
for i in range(len(lab)):
    if lab[i]==output_classs:
        print(names[i])

In [0]:
print(type(data['Volleys'].values))
print(data['Volleys'].values)
print(type(data))

# Fifa 20 

In [6]:
data = pd.read_csv('/content/gdrive/My Drive/Projects/Fifa19/players_20.csv', delimiter=',', header=0, index_col=0)
print(data.head())

                                                  player_url  ...    rb
sofifa_id                                                     ...      
158023     https://sofifa.com/player/158023/lionel-messi/...  ...  63+2
20801      https://sofifa.com/player/20801/c-ronaldo-dos-...  ...  61+3
190871     https://sofifa.com/player/190871/neymar-da-sil...  ...  61+3
200389     https://sofifa.com/player/200389/jan-oblak/20/...  ...   NaN
183277     https://sofifa.com/player/183277/eden-hazard/2...  ...  61+3

[5 rows x 103 columns]


In [40]:
# for i,col in zip(range(len(data.columns)),data.columns):
#   print (i, col)
print([(i,col) for (i,col) in zip(range(len(data.columns)),data.columns)])
# list(data.columns)

[(0, 'player_url'), (1, 'short_name'), (2, 'long_name'), (3, 'age'), (4, 'dob'), (5, 'height_cm'), (6, 'weight_kg'), (7, 'nationality'), (8, 'club'), (9, 'overall'), (10, 'potential'), (11, 'value_eur'), (12, 'wage_eur'), (13, 'player_positions'), (14, 'preferred_foot'), (15, 'international_reputation'), (16, 'weak_foot'), (17, 'skill_moves'), (18, 'work_rate'), (19, 'body_type'), (20, 'real_face'), (21, 'release_clause_eur'), (22, 'player_tags'), (23, 'team_position'), (24, 'team_jersey_number'), (25, 'loaned_from'), (26, 'joined'), (27, 'contract_valid_until'), (28, 'nation_position'), (29, 'nation_jersey_number'), (30, 'pace'), (31, 'shooting'), (32, 'passing'), (33, 'dribbling'), (34, 'defending'), (35, 'physic'), (36, 'gk_diving'), (37, 'gk_handling'), (38, 'gk_kicking'), (39, 'gk_reflexes'), (40, 'gk_speed'), (41, 'gk_positioning'), (42, 'player_traits'), (43, 'attacking_crossing'), (44, 'attacking_finishing'), (45, 'attacking_heading_accuracy'), (46, 'attacking_short_passing'), 

In [36]:
from operator import itemgetter
itemgetter(0,5,10,12)(data.columns)

('player_url', 'height_cm', 'potential', 'wage_eur')